## Regresión multiple

In [1]:
import os
import json
import pandas as pd

from sklearn.metrics import mean_squared_error, r2_score
from stats_utils.regression_models import simple_regression

In [2]:
path = r"C:\Users\DELL\PycharmProjects\Tesis\Parcelas\Parcelas_2.geojson"
df_path = r"C:\Users\DELL\PycharmProjects\Tesis\dataframes\Evapo"

In [14]:
ndvi_promedio = {}

for i in os.listdir(r"C:\Users\DELL\PycharmProjects\Tesis\dataframes\Evapo"):
    df = pd.read_csv(r"C:\Users\DELL\PycharmProjects\Tesis\dataframes\Evapo\{}".format(i))
    df['Fecha'] = df['Fecha'].astype('datetime64[ns]')
    df = df.sort_values(by='Fecha', ascending=True)
    file_name, ext = i.split('.')
    parcela_id = file_name.split('_')[1]

    with open(path, encoding='utf-8') as par:
        parcelas = json.load(par)
        for j in parcelas['features']:
            if int(parcela_id) == j['properties']['Id']:
                # print(parcela_id, j['properties']['Rendimiento'])
                ndvi_promedio[parcela_id] = (df['ndvi_mean'].mean(), df['ndmi_mean'].mean(), df['msi_mean'].mean(), df['ETc'].mean(), j['properties']['Rendimiento'], int(parcela_id))

ndvi_yield_df = pd.DataFrame.from_dict(ndvi_promedio, orient='index', columns=['ndvi', 'ndwi', 'msi', 'etc', 'yield', 'id'])
print(ndvi_yield_df.corr(method='pearson'))
x = ndvi_yield_df[['ndvi', 'ndwi', 'msi', 'etc']]
y = ndvi_yield_df['yield']
model = simple_regression(x, y)
equation = 'y = {}x + ({})'.format(model.coef_, model.intercept_)
y_pred = model.predict(x)
ndvi_yield_df['predict'] = y_pred
print(f"Coeficientes: {model.coef_}")
print(f"Intercepto: {model.intercept_}")
print(f"Error cuadrático medio: {mean_squared_error(y, y_pred)}")
print(f"R2: {r2_score(y, y_pred)}")
print(model.score(x, y))
print(ndvi_yield_df.sort_values(by='id', ascending=True))

           ndvi      ndwi       msi       etc     yield        id
ndvi   1.000000  0.937763 -0.893715  0.732119  0.836061  0.062668
ndwi   0.937763  1.000000 -0.985034  0.693076  0.807362  0.080969
msi   -0.893715 -0.985034  1.000000 -0.674435 -0.721653 -0.184480
etc    0.732119  0.693076 -0.674435  1.000000  0.370054 -0.042500
yield  0.836061  0.807362 -0.721653  0.370054  1.000000 -0.244834
id     0.062668  0.080969 -0.184480 -0.042500 -0.244834  1.000000
Coeficientes: [ 237.24359125 1055.41017549  471.38581635  -35.80908623]
Intercepto: -439.86824527731835
Error cuadrático medio: 18.628687682477043
R2: 0.9344941904704571
0.9344941904704571
        ndvi      ndwi       msi       etc  yield  id    predict
1   0.509355  0.001981  1.056740  3.849733     48   1  43.340145
2   0.536524  0.084066  0.898189  4.031022     60   2  55.189265
3   0.516467  0.071127  0.897449  3.907103     45   3  40.863317
4   0.516839  0.056572  0.961190  3.915978     50   4  55.319465
5   0.556311  0.100697  